In [1]:
import intake_tools
import intake
from pystac import Catalog, Item, Asset, MediaType, CatalogType
import stac_tools
import logging

logger = logging.getLogger("PlayWithStack")

logger.setLevel(logging.ERROR)

In [2]:
cat = intake.open_catalog("../catalog.yaml")

In [3]:
stac_cat = Catalog("nextgems", "nextGEMS simulations")


def create_subcat(cat, child, position):
    parent = stac_tools.get_subcat(stac_cat, position)
    child = Catalog(child, f"Sub-catalog for {'/'.join(position)}/{child}")
    parent.add_child(child)


def create_item(cat, child, position):
    try:
        create_zarr_item(cat, child, position)
    except Exception as e:
        logger.error(f"Bad dataset: {'/'.join(position)}/{child}: {e}")


def create_zarr_item(cat, child, position):
    logger.debug(f'processing "/".join(position + [child])')

    parent = stac_tools.get_subcat(stac_cat, position)
    cg = intake_to_stac.catalog_generator()
    child = cg._gen_item(child, cat[child])
    parent.add_item(child)

In [4]:
def print_subcat(cat, child, position):
    print("/".join(position + [child]))


def print_entry(cat, child, position):
    print("/".join(position + [child]) + ".yaml")


In [5]:
%load_ext autoreload
%aimport intake_to_stac
%autoreload 1


In [6]:
intake_tools.traverse_tree(
    cat["ICON.PrecipPuzzle"], subcat_callback=create_subcat, entry_callback=create_item
)

In [7]:
stac_cat.normalize_and_save(
    root_href="/tmp/stac", catalog_type=CatalogType.SELF_CONTAINED
)